In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import cohere
from dotenv import load_dotenv
import os
import nltk
nltk.download('punkt')  # Download the tokenizer models

from nltk.tokenize import word_tokenize
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yunethirikhin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from the environment
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
load_dotenv()

# Retrieve the API key from the environment
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


#client_gpt = OpenAI(
#  api_key=OPENAI_API_KEY,  # this is also the default, it can be omitted
#)

True

In [3]:
co = cohere.ClientV2(COHERE_API_KEY)
client = QdrantClient(url="http://localhost:6333")
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='aapl_10k_10q_forms'), CollectionDescription(name='earnings_calls'), CollectionDescription(name='financial_news')])

In [4]:
collection_name = "earnings_calls"
collection_info = client.get_collection(collection_name)

print(f"Collection info for {collection_name}:")
print(collection_info)

Collection info for earnings_calls:
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=48 segments_count=4 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segment

In [4]:
client.create_collection(
    collection_name="summarizer_test3",
    vectors_config=VectorParams(size=1024, distance=Distance.DOT),
)

True

In [ ]:
file_path = '/home/sarahgohrazer/Projects/Stocks-MultiAgent/aapl_10k_forms/apple_filings_text/000032019324000123-aapl-20240928.txt' 
with open(file_path, 'r') as file:
    text = file.read()

# Divide the text into chunks
chunk_size = 3000  # Adjust this size as needed
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

In [47]:
model="embed-english-v3.0"

doc_embeddings = co.embed(texts=chunks,
                          model=model,
                          input_type="search_document",
                          embedding_types=['float'])

NameError: name 'chunks' is not defined

In [5]:
query ="""
WHat 
"""

In [22]:
points = []
for idx, (embedding, doc) in enumerate(zip(doc_embeddings.embeddings.float_, chunks)):
    point = PointStruct(
        id=idx,
        vector=embedding,
        payload={"document": doc}
    )
    points.append(point)

In [11]:
operation_info = client.upsert(
    collection_name="summarizer_test3",
    points=points
)

In [6]:
model="embed-english-v3.0"

query_embeddings = co.embed(texts=[query],
                          model=model,
                          input_type="search_query",
                          embedding_types=['float'])

In [17]:
search_result = client.query_points(
    collection_name="financial_news", query=query_embeddings.embeddings.float_[0], limit=10
).points

In [19]:
for hit in search_result:
    print(hit.payload.keys())
    print(hit.payload['sentiment'])
    print(f"ID: {hit.id}, Document: {hit.payload['document']}, Score: {hit.score}")

dict_keys(['document', 'file_name', 'chunk_index', 'title', 'source', 'summary', 'publish_date', 'url', 'sentiment', 'confidence'])
neutral
ID: 4527387593854241285, Document: The "Magnificent Seven" — made up of Nvidia (NVDA), Alphabet (GOOG, GOOGL), Tesla (TSLA), Microsoft (MSFT), Amazon (AMZN), Meta Platforms (META), and Apple (AAPL) — drove major indexes (^DJI, ^IXIC, ^GSPC) higher in 2024. Niles Investment Management founder and portfolio manager Dan Niles joins Market Domination Co-Hosts Julie Hyman and Josh Lipton to discuss how to play the megacap tech group in the new year., Score: 0.36994368
dict_keys(['document', 'file_name', 'chunk_index', 'title', 'source', 'summary', 'publish_date', 'url', 'sentiment', 'confidence'])
positive
ID: 3972119060730086807, Document: There's arguably not a billionaire money manager that garners more attention on Wall Street than Berkshire Hathaway's (NYSE: BRK.A)(NYSE: BRK.B) chief, Warren Buffett. The accurately dubbed "Oracle of Omaha" has led 

In [14]:
document_list = [point.payload['document'] for point in search_result]

document_list

['umber, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class\nTrading symbol(s)\nName of each exchange on which registered\nCommon Stock, $0.00001 par value per share\nAAPL\nThe Nasdaq Stock Market LLC\n0.000% Notes due 2025\n—\nThe Nasdaq Stock Market LLC\n0.875% Notes due 2025\n—\nThe Nasdaq Stock Market LLC\n1.625% Notes due 2026\n—\nThe Nasdaq Stock Market LLC\n2.000% Notes due 2027\n—\nThe Nasdaq Stock Market LLC\n1.375% Notes due 2029\n—\nThe Nasdaq Stock Market LLC\n3.050% Notes due 2029\n—\nThe Nasdaq Stock Market LLC\n0.500% Notes due 2031\n—\nThe Nasdaq Stock Market LLC\n3.600% Notes due 2042\n—\nThe Nasdaq Stock Market LLC\nSecurities registered pursuant to Section 12(g) of the Act:  None\nIndicate by check mark if the Registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.\nYes\n☒\nNo\n☐\nIndicate by check mark if the Registrant is not required to file reports pursuant to Section 13 or Sect

checking token limit, max 4096 tokens in each document

In [16]:
# Assuming document_list contains text-based documents (strings)
tokens_per_document = [len(word_tokenize(doc)) for doc in document_list]

# Printing the number of tokens for each document
print(tokens_per_document)

## max 4096 tokens in each document 

[586, 538, 256, 540, 269, 518, 274, 283, 256, 375]


In [17]:
rerank_results = co.rerank(
    model="rerank-english-v3.0",
    query=query,
    documents=document_list,
    top_n=3,
    return_documents=True
)

In [18]:
# print(dir(rerank_results))
top_documents = [item.document.text for item in rerank_results.results]
print(top_documents)
print(len(top_documents))

['umber, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class\nTrading symbol(s)\nName of each exchange on which registered\nCommon Stock, $0.00001 par value per share\nAAPL\nThe Nasdaq Stock Market LLC\n0.000% Notes due 2025\n—\nThe Nasdaq Stock Market LLC\n0.875% Notes due 2025\n—\nThe Nasdaq Stock Market LLC\n1.625% Notes due 2026\n—\nThe Nasdaq Stock Market LLC\n2.000% Notes due 2027\n—\nThe Nasdaq Stock Market LLC\n1.375% Notes due 2029\n—\nThe Nasdaq Stock Market LLC\n3.050% Notes due 2029\n—\nThe Nasdaq Stock Market LLC\n0.500% Notes due 2031\n—\nThe Nasdaq Stock Market LLC\n3.600% Notes due 2042\n—\nThe Nasdaq Stock Market LLC\nSecurities registered pursuant to Section 12(g) of the Act:  None\nIndicate by check mark if the Registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.\nYes\n☒\nNo\n☐\nIndicate by check mark if the Registrant is not required to file reports pursuant to Section 13 or Sect

send to open ai for query

In [20]:
# Combine the top documents into a single prompt
combined_documents = "\n\n".join(top_documents[:3])  # Use top 3 or however many you want

# Prepare the prompt for the combined documents
prompt = f"Query: {query}\n\nDocuments: {combined_documents}\n\nAnswer:"

# Generate a response using the OpenAI GPT model
response = client_gpt.responses.create(
    model="gpt-3.5-turbo",  # Use the desired OpenAI model
    input=prompt,  # The prompt for the LLM
)

# Output the answer
print(response.output_text.strip())


The ticker symbol of the company mentioned in the document is AAPL.
